In [1]:
import pandas as pd
import numpy as np
import fastf1
import datetime
fastf1.Cache.enable_cache('cache') 

In [2]:
def find_fastest_laps(laps, cols, index_cols, lap_num):
    
    laps = laps.sort_values('LapTime')
    fastest = laps.groupby('DriverNumber').nth(0).reset_index()

    fastest_time = {}
    added_cols = []
    
    
    for col in cols:
        fastest_time[col] = laps[col].min()
        
        added_cols.append(col[:-4] + 'Percent')
        added_cols.append(col[:-4] + 'Position')
        # fastest[col[:-4] + 'Percent'] = (fastest[col] - fastest_time) / fastest_time
        # total_cols.append(col[:-4] + 'Percent')
    
    # dummies = pd.get_dummies(fastest['Compound'])
    # #dummies.columns = dummies.columns + '_1'
    # fastest = pd.concat([fastest, dummies], axis = 1)
    
    fastest = fastest[index_cols]
    

    # top_cols = ['Sector1Time', 'Sector2Time', 'Sector3Time']
    # fastest = fastest.merge(laps.groupby('DriverNumber').min()[top_cols].reset_index(), on=['DriverNumber'], how = 'outer')
    for i in range(0,lap_num):
        laps_i = laps.groupby('DriverNumber').nth(i).reset_index()

        for col in ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']:
            laps_i[col[:-4] + 'Percent'] = (laps_i[col] - fastest_time[col]) / fastest_time[col]
            laps_i[col[:-4] + 'Position'] = laps_i.sort_values(col).index
        # laps_i['LapPercent'] = (laps_i['LapTime'] - fastest_lap) / fastest_lap
        fastest = fastest.merge(laps_i[index_cols + cols + added_cols], on=index_cols, how = 'outer', suffixes=('', '_' + str(i)))
    return fastest

In [3]:
def load_practice(event, type, year):
    
    practice1 = fastf1.get_session(year, event, 'Practice 1')
    practice1.load(laps=True)
    try:
        laps1 = practice1.laps
        cols = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
        index_cols = ['DriverNumber', 'Team', 'Driver']
        practice1 = find_fastest_laps(laps1, cols, index_cols, 2)
        practice2 = practice1
        if type == 'conventional':
            practice2 = fastf1.get_session(year, event, 'Practice 2')
            practice2.load(laps=True)
            laps2 = practice2.laps
            practice2 = find_fastest_laps(laps2, cols, index_cols, 2)

        practice_data = practice1.merge(practice2, on= index_cols, how = 'outer', suffixes=('_1', '_2'))
        return practice_data
    except:
         return pd.DataFrame()
   
def load_qualifying(event, year):
        qualifying = fastf1.get_session(year, event, 'Q')
        qualifying.load()
        try:
            results = qualifying.results

            results['FastestLap'] = results[['Q1', 'Q2', 'Q3']].apply(lambda x: min(x), axis = 1)

            fastest_lap_q = results['FastestLap'].min()
            results['LapPercent'] = (results['FastestLap'] - fastest_lap_q)/ fastest_lap_q
            results['Team'] = results['TeamName']
            return results[['DriverNumber', 'Position', 'LapPercent', 'Team']]
        except:
             return pd.DataFrame()


In [4]:
years = [2018, 2019,2020, 2021, 2022, 2023]
years = [2021, 2022]

data = pd.DataFrame()
#prev_qualifying_data = pd.DataFrame()
for year in years:
    schedule = fastf1.get_event_schedule(year)
    for index, row in schedule.iterrows():
        if row['EventFormat'] in ['conventional']:
            practice_data = load_practice(row['EventName'], row['EventFormat'], year)
            qualifying_data = load_qualifying(row['EventName'], year)
            if 'DriverNumber' in practice_data.columns and 'DriverNumber' in qualifying_data.columns:
                #if 'DriverNumber' in prev_qualifying_data.columns:
                    # qualifying_data_full = qualifying_data.merge(prev_qualifying_data, on = 'DriverNumber', suffixes=('_curr', '_prev'))
                full_data = practice_data.merge(qualifying_data, on=['DriverNumber', 'Team'])
                for col in ['RoundNumber', 'Country', 'Location', 'OfficialEventName', 'EventDate','EventName', 'EventFormat']:
                    full_data[col] = row[col]
                data = pd.concat([data, full_data], axis = 0, ignore_index=True)
                # prev_qualifying_data = qualifying_data

core           INFO 	Loading data for Bahrain Grand Prix - Practice 1 [v2.3.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1492: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for key, value in row.iteritems():
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packag

In [5]:
data = pd.DataFrame(data)
prev_range = 5

In [6]:
for i in range(prev_range):
    data['PrevLapPercent' + str(i+1)] = data.groupby('DriverNumber')['LapPercent'].shift(i+1)
    data['PrevPosition' + str(i+1)] = data.groupby('DriverNumber')['Position'].shift(i+1)
    prev = data.groupby(['Team', 'OfficialEventName', 'EventDate'])[['Position', 'LapPercent']].mean().reset_index().sort_values('EventDate').groupby('Team').shift(1)
    prev.index = data.groupby(['Team', 'OfficialEventName', 'EventDate'])[['Position', 'LapPercent']].mean().reset_index(2).sort_values('EventDate').index
    prev = prev[['Position', 'LapPercent']].reset_index()
    data = data.merge(prev, on = ['OfficialEventName', 'Team'], suffixes=('', 'PrevTeam' + str(i+1)))

KeyError: 'Team'

In [ ]:
data.to_pickle('train_data_test.pkl')

In [ ]:
sched_2023 = fastf1.get_event_schedule(2023)
round_number = 5

In [ ]:
pred_event = sched_2023[sched_2023['RoundNumber'] == round_number]

In [ ]:
pred_data = load_practice(pred_event['EventName'].iloc[0], pred_event['EventFormat'].iloc[0], 2023)
for col in ['RoundNumber', 'Country', 'Location', 'OfficialEventName', 'EventDate','EventName', 'EventFormat']:
    pred_data[col] = pred_event[col].iloc[0]
for i in range(prev_range):
    prev_event = sched_2023[sched_2023['RoundNumber'] == round_number - i - 1]
    prev_data = load_qualifying(prev_event['EventName'].iloc[0], 2023)
    prev_data['PrevLapPercent' + str(i+1)] = prev_data['LapPercent']
    prev_data['PrevPosition' + str(i+1)] = prev_data['Position']
    prev_data = prev_data.merge(prev_data.groupby('Team')['LapPercent', 'Position'].mean().reset_index(), on=['Team'], suffixes=('', 'PrevTeam' + str(i+1)))

    pred_data = pred_data.merge(prev_data[['DriverNumber', 'PrevLapPercent' + str(i+1),'PrevPosition' + str(i+1), 'LapPercentPrevTeam' + str(i+1), 'PositionPrevTeam' + str(i+1)]], on = 'DriverNumber')

core           INFO 	Loading data for Miami Grand Prix - Practice 1 [v2.3.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1492: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for key, value in row.iteritems():
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages

Index(['DriverNumber', 'Position', 'LapPercent', 'Team'], dtype='object')


/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1492: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for key, value in row.iteritems():
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1492: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items inst

Index(['DriverNumber', 'Position', 'LapPercent', 'Team'], dtype='object')


api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '16', '14', '63', '55', '18', '31', '44', '81', '10', '27', '24', '20', '77', '1', '22', '23', '21', '4', '2']


Index(['DriverNumber', 'Position', 'LapPercent', 'Team'], dtype='object')


/var/folders/xb/k6q1hgwx19dgzl829cdrb0500000gn/T/ipykernel_92433/1325109791.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_data['PrevLapPercent' + str(i+1)] = prev_data['LapPercent']
/var/folders/xb/k6q1hgwx19dgzl829cdrb0500000gn/T/ipykernel_92433/1325109791.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev_data['PrevPosition' + str(i+1)] = prev_data['Position']
/var/folders/xb/k6q1hgwx19dgzl829cdrb0500000gn/T/ipykernel_92433/1325109791.py:10: FutureWarning: Indexing with multiple keys (imp

In [ ]:
pred_data.to_pickle('pred_data.pkl')